# MLB.com stats crawler
https://github.com/whyhugo/Data-Analysis-in-Sports
> Please input the URL which can get from https://www.mlb.com/stats

In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [2]:

chrome_options = Options()
chrome_options.add_argument('--headless')



url = input('URL:')
driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
html = driver.get(url)
sp = soup(driver.page_source, 'html.parser')

C:\Users\user\AppData\Local\Temp\ipykernel_18992\654874124.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


get player index
```python
index_list = []
index = sp.find_all(class_ = 'index-3cdMSKi7')
for i in range(0, len(index), 2):
    index_list.append(index[i].text)
```

In [3]:
# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

In [4]:
# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

In [5]:
def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

In [6]:
# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [7]:
# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(3)

In [8]:
# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [9]:
# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

```python
html = driver.get(url) 
# 有時候會 get 不完整，可以用這邊 reload 一次 XD
```

In [10]:
try:
    df_list = []
    while True:
        sp = soup(driver.page_source, 'html.parser')
        print('page start')
        time.sleep(2)
        name = get_player_name()
        time.sleep(2)
        position = get_player_position()
        time.sleep(2)
        current_data = get_data()
        time.sleep(2)
        for i in range(len(name)):
            row = []
            row.append(name[i])
            row.append(position[i])
            row.extend(current_data[i*32:i*32+32])
            df_list.append(row)
        click_next_page_button()
except:
    print('This is the last page :)')

page start
page start
This is the last page :)


In [11]:
df = pd.DataFrame(df_list, columns=get_column_name())
df

,PLAYER,POSITION,TEAM,G,AB,R,H,2B,3B,HR,...,GO/AO,XBH,TB,IBB,BABIP,ISO,AB/HR,BB/K,BB%,SO%
0,John Hester,C,LAA,1,0,1,0,0,0,0,...,-.--,0,0,0,.---,--,-.--,.---,1.000,.000
1,Mike Trout,CF,LAA,157,589,109,190,39,9,27,...,0.83,75,328,10,.376,.234,21.82,.809,.154,.190
2,Kole Calhoun,RF,LAA,58,195,29,55,7,2,8,...,1.04,17,90,0,.311,.179,24.38,.512,.095,.185
3,Howie Kendrick,2B,LAA,122,478,55,142,21,4,13,...,1.66,38,210,5,.340,.142,36.77,.258,.045,.173
4,Albert Pujols,DH,LAA,99,391,49,101,19,0,17,...,0.78,36,171,8,.258,.179,23.00,.727,.090,.124
5,Efren Navarro,1B,LAA,4,4,0,1,0,0,0,...,2.00,0,1,0,.333,.000,-.--,2.000,.333,.167
6,Mark Trumbo,1B,LAA,159,620,85,145,30,2,34,...,1.17,66,281,6,.273,.219,18.24,.293,.080,.271
7,Josh Hamilton,RF,LAA,151,576,73,144,32,5,21,...,0.81,58,249,4,.303,.182,27.43,.297,.074,.248
8,Chris Iannetta,C,LAA,115,325,40,73,15,0,11,...,0.71,26,121,2,.284,.148,29.54,.680,.170,.251
9,Grant Green,2B,LAA,40,125,16,35,8,1,1,...,0.83,10,48,0,.391,.104,125.00,.263,.073,.277


In [12]:
df.to_csv('mlb_stats___.csv', index=False, encoding='utf-8')

In [ ]:
driver.quit()